<a href="https://colab.research.google.com/github/mpampin/car-model-classifier-cnn/blob/master/Chapadany_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from google.colab import files

In [0]:
maxlen = 100
embedding_dims = 50
filters = 256
kernel_size = 3
hidden_dims = 128
epochs = 20

In [3]:
uploaded = files.upload()


Saving cromosol-data.csv to cromosol-data.csv


In [0]:
dataset = pd.read_csv('cromosol-data.csv')
dataset

(24329, 6)

In [0]:
X = dataset.iloc[:, 5].values
y = dataset.iloc[:, 1].values
X, y

(array(['CASCO OPTICA DODGE P.UP D-100 56/61 (BASE)',
        'RETEN OPTICA FIAT S.EUROPA 83/91',
        'RETEN OPTICA FIAT S.EUROPA 83/91', ...,
        'REP.MAQ.L/C ELECT.PEUG.307 01/06 TRAS.(COMPL.)',
        'PORTON FORD RANGER 12/15 TRAS.',
        'CANALIZADOR AIRE CHEV.ONIX 13/17'], dtype=object),
 array(['DODGE|DODGE|DODGE', 'FIAT|FIAT', 'FIAT', ..., 'PEUGEOT|PEUGEOT',
        'FORD|FORD',
        'CHEVROLET|CHEVROLET|CHEVROLET|CHEVROLET|CHEVROLET|CHEVROLET|CHEVROLET|CHEVROLET|CHEVROLET'],
       dtype=object))

In [0]:
from scipy.stats import mode
#tmp = []
#for data in y:
#  out = []
#  trademarks = data[0].split("|")
#  models = data[1].split("|")
#  for i in range(len(trademarks)):
#    out.append(trademarks[i] + " " + models[i])
  
#  tmp.append(out)

y = np.array([mode(out.split("|"))[0][0] for out in y])
y

array(['DODGE', 'FIAT', 'FIAT', ..., 'PEUGEOT', 'FORD', 'CHEVROLET'],
      dtype='<U13')

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
y_ct = MultiLabelBinarizer()
encoded_y = y_ct.fit_transform(y.reshape((y.shape[0], 1)))
encoded_y.shape

(24329, 41)

In [0]:
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
encoded_X = np.array([np.array([ord(char) if char.isalnum() else 32 for char in list(data)]) for data in X])
encoded_X = sequence.pad_sequences(encoded_X, maxlen=maxlen)
shape = encoded_X.shape

#x_ct = LabelEncoder()
#encoded_X = x_ct.fit_transform(encoded_X.reshape(shape[0] * shape[1]))


#encoded_X = encoded_X.reshape(shape)
encoded_X

array([[ 0,  0,  0, ..., 83, 69, 32],
       [ 0,  0,  0, ..., 32, 57, 49],
       [ 0,  0,  0, ..., 32, 57, 49],
       ...,
       [ 0,  0,  0, ..., 76, 32, 32],
       [ 0,  0,  0, ..., 65, 83, 32],
       [ 0,  0,  0, ..., 32, 49, 55]], dtype=int32)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(1000,
                    64,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(64,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# Another vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(encoded_y.shape[1]))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encoded_X, encoded_y, test_size = 0.1, random_state = 0)

In [0]:
model.fit(X_train, y_train,
          batch_size=16,
          epochs=epochs,
          validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21896 samples, validate on 2433 samples
Epoch 1/20
21896/21896 [==============================] - 10s 464us/step - loss: 1.4353 - accuracy: 0.5877 - val_loss: 0.8212 - val_accuracy: 0.7707
Epoch 2/20
21896/21896 [==============================] - 10s 468us/step - loss: 0.7516 - accuracy: 0.7948 - val_loss: 0.6678 - val_accuracy: 0.8138
Epoch 3/20
21896/21896 [==============================] - 10s 450us/step - loss: 0.6496 - accuracy: 0.8193 - val_loss: 0.6571 - val_accuracy: 0.8266
Epoch 4/20
21896/21896 [==============================] - 10s 444us/step - loss: 0.6113 - accuracy: 0.8261 - val_loss: 0.6265 - val_accuracy: 0.8290
Epoch 5/20
21896/21896 [==============================] - 10s 448us/step - loss: 0.5826 - accuracy: 0.8333 - val_loss: 0.6207 - val_accuracy: 0.8352
Epoch 6/20
21896/21896 [==============================] - 10s 452us/step - loss: 0.5603 - accuracy: 0.8375 - val_loss: 0.6018 - val_accuracy: 0.8434
Epoch 7/20
21896/21896 [==============================] -

In [0]:
test_string = "OPTICA DELANTERA DEL FIAT 600 500 y FIORINO"

encoded_test = np.array([[ord(char) if char.isalnum() else 32 for char in list(test_string)]])
encoded_test = sequence.pad_sequences(encoded_test, maxlen=maxlen)

prediction = model.predict(encoded_test)
prediction = np.array([round(d) for d in prediction[0]])

test_string, y_ct.inverse_transform(np.array([prediction])), prediction

('OPTICA DELANTERA DEL FIAT 600 500 y FIORINO',
 [('FIAT',)],
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [0]:
X[102]

'SOPORTE PARAG.DEL.FORD FALCON 78/81 EXT.'